In [ ]:
library("car")
data("Leinhardt")
head(Leinhardt)

In [ ]:
pairs(Leinhardt)

In [ ]:
plot(infant ~ income, data = Leinhardt)

In [ ]:
Leinhardt$loginfant = log(Leinhardt$infant)
Leinhardt$logincome = log(Leinhardt$income)
plot(loginfant ~ logincome, data = Leinhardt)

In [ ]:
lmod = lm(loginfant ~ logincome, data = Leinhardt)
summary(lmod)

In [ ]:
dat = na.omit(Leinhardt)
library("rjags")

In [ ]:
mod1_str = "model {
    for (i in 1: n) {
        y[i] ~ dnorm(mu[i], prec)
        mu[i] = b[1] + b[2] * log_income[i]        
    }
    
    for (i in 1:2) {
        b[i] ~ dnorm(0.0, 1./1.e6)        
    }
    
    prec ~ dgamma(5/2.0, 5*10/2.0)
    sig2 = 1 / prec
    sig = sqrt(sig2)
}"

set.seed(42)
data1_jags = list(y=dat$loginfant, n=nrow(dat), log_income=dat$logincome)
params1 = c("b", "sig")

inits1 = function() { 
    inits = list("b" = rnorm(2, 0.0, 100.), "prec"=rgamma(1, 1.0, 1.0))
}

mod1 = jags.model(textConnection(mod1_str), data = data1_jags, inits = inits1, n.chains = 3)
update(mod1, 1000) # burn in

mod1_sim = coda.samples(model = mod1, variable.names = params1, n.iter = 5000)
mod1_csim = do.call(rbind, mod1_sim)

In [ ]:
plot(mod1_sim)

In [ ]:
autocorr.plot(mod1_sim)

In [ ]:
effectiveSize(mod1_sim)

## Residuals checks

In [ ]:
lmod0 = lm(infant ~ income, data=Leinhardt)
plot(resid(lmod0)) # to check independence (looks okay

In [ ]:
plot(predict(lmod0), resid(lmod0)) # to check for linearity, constant variance (looks bad)

In [ ]:
qqnorm(resid(lmod0)) # to check Normality assumption (we want this to be a straight line)

## Bayesian residuals

In [ ]:
X = cbind(rep(1.0, data1_jags$n), data1_jags$log_income)
head(X)

In [ ]:
pm_params1 = colMeans(mod1_csim)

In [ ]:
yhat1 = drop(X %*% pm_params1[1:2]) # drop makes an array
resid1 = data1_jags$y - yhat1
plot(resid1)

In [ ]:
plot(yhat1, resid1)

In [ ]:
qqnorm(resid1)

In [ ]:
plot(predict(lmod), resid(lmod))

## Adding additional covariates

In [ ]:
library("rjags")

mod2_string = " model {
    for (i in 1:length(y)) {
        y[i] ~ dnorm(mu[i], prec)
        mu[i] = b[1] + b[2]*log_income[i] + b[3]*is_oil[i]
    }
    
    for (i in 1:3) {
        b[i] ~ dnorm(0.0, 1.0/1.0e6)
    }
    
    prec ~ dgamma(5/2.0, 5*10.0/2.0)
    sig = sqrt( 1.0 / prec )
} "


set.seed(73)
data2_jags = list(y=dat$loginfant, log_income=dat$logincome,
                  is_oil=as.numeric(dat$oil=="yes"))
data2_jags$is_oil

params2 = c("b", "sig")

inits2 = function() {
    inits = list("b"=rnorm(3,0.0,100.0), "prec"=rgamma(1,1.0,1.0))
}

mod2 = jags.model(textConnection(mod2_string), data=data2_jags, inits=inits2, n.chains=3)
update(mod2, 1e3) # burn-in

mod2_sim = coda.samples(model=mod2,
                        variable.names=params2,
                        n.iter=5e3)

mod2_csim = as.mcmc(do.call(rbind, mod2_sim)) # combine multiple chains

In [ ]:
plot(mod2_sim)

In [ ]:
X2 = cbind(rep(1.0, data1_jags$n), data2_jags$log_income, data2_jags$is_oil)
head(X2)

In [ ]:
(pm_params2 = colMeans(mod2_csim)) # posterior mean

In [ ]:
yhat2 = drop(X2 %*% pm_params2[1:3])
resid2 = data2_jags$y - yhat2
plot(resid2) # against data index

In [ ]:
plot(yhat2, resid2) # against predicted values


## t-distribution

In [ ]:
mod3_string = " model {
    for (i in 1:length(y)) {
        y[i] ~ dt( mu[i], tau, df )
        mu[i] = b[1] + b[2]*log_income[i] + b[3]*is_oil[i]
    }
    
    for (i in 1:3) {
        b[i] ~ dnorm(0.0, 1.0/1.0e6)
    }
    
    nu ~ dexp(1.0)    
    df = nu + 2.0 # we want degrees of freedom > 2 to guarantee existence of mean and variance
    tau ~ dgamma(5/2.0, 5*10.0/2.0) # tau is close to, but not equal to the precision
    sig = sqrt( 1.0 / tau * df / (df - 2.0) ) # standard deviation of errors
} "

set.seed(73)
data3_jags = list(y=dat$loginfant, log_income=dat$logincome,
                  is_oil=as.numeric(dat$oil=="yes"))

params3 = c("b")

inits3 = function() {
    inits = list("b"=rnorm(3,0.0,100.0))
}

mod3 = jags.model(textConnection(mod3_string), data=data3_jags, inits=inits3, n.chains=3)
update(mod3, 1e3) # burn-in

mod3_sim = coda.samples(model=mod3,
                        variable.names=params3,
                        n.iter=5e3)

mod3_csim = as.mcmc(do.call(rbind, mod3_sim)) # combine multiple chains

In [ ]:
plot(mod3_csim)

summary(mod2_sim)

gelman.diag(mod3_sim)

In [ ]:
X3 = cbind(rep(1.0, data1_jags$n), data2_jags$log_income, data2_jags$is_oil)
head(X3)

In [ ]:
pm_params3 = colMeans(mod3_csim)
pm_params3

In [ ]:
yhat3 = X3 %*% pm_params3
resid3 = data3_jags$y - yhat3

In [ ]:
plot(resid3)

In [ ]:
dic.samples(mod1,n.iter = 1e3)

In [ ]:
dic.samples(mod2,n.iter = 1e3)

In [ ]:
dic.samples(mod3,n.iter = 1e3)

## Assignments

In [ ]:
library("car")  # load the 'car' package
data("Anscombe")  # load the data set
?Anscombe  # read a description of the data
head(Anscombe) 
pairs(Anscombe)

In [ ]:
lmod = lm(education ~ income + young + urban,data = Anscombe)
summary(lmod)

In [ ]:
plot(lmod)d

In [ ]:
library("rjags")

mod_string = " model {
    for (i in 1:length(education)) {
        education[i] ~ dnorm(mu[i], prec)
        mu[i] = b0 + b[1]*income[i] + b[2]*young[i] + b[3]*urban[i]
    }
    
    b0 ~ dnorm(0.0, 1.0/1.0e6)
    for (i in 1:3) {
        b[i] ~ dnorm(0.0, 1.0/1.0e6)
    }
    
    prec ~ dgamma(1.0/2.0, 1.0*1500.0/2.0)
    ## Initial guess of variance based on overall
    ## variance of education variable. Uses low prior
    ## effective sample size. Technically, this is not
    ## a true 'prior', but it is not very informative.
    sig2 = 1.0 / prec
    sig = sqrt(sig2)
} "

data_jags = as.list(Anscombe)

params = c("b") # define model params to estimate 

inits = function() { # inits to get function 
    inits = list("b"=rnorm(3,0.0, 100.0))
}

mod = jags.model(textConnection(mod_string), data=data_jags, inits=inits, n.chains=3)
update(mod, 1e4) # burn-in
mod_sim = coda.samples(model=mod,variable.names=params,n.iter=1e5)
mod_csim = as.mcmc(do.call(rbind, mod_sim)) # combine multiple chains
dic.samples(mod, n.iter = 1e5)

In [ ]:
plot(mod_sim)

In [ ]:
mod_csim['b']

In [ ]:
typeof(mod_csim)

In [ ]:
colMeans(mod_csim)

In [ ]:
mean(mod_sim[ ,3] > 0)

In [ ]:
library("rjags")

mod1_string = " model {
    for (i in 1:length(education)) {
        education[i] ~ dnorm(mu[i], prec)
        mu[i] = b0 + b[1]*income[i] + b[2]*young[i]
    }
    
    b0 ~ dnorm(0.0, 1.0/1.0e6)
    for (i in 1:2) {
        b[i] ~ dnorm(0.0, 1.0/1.0e6)
    }
    
    prec ~ dgamma(1.0/2.0, 1.0*1500.0/2.0)
    ## Initial guess of variance based on overall
    ## variance of education variable. Uses low prior
    ## effective sample size. Technically, this is not
    ## a true 'prior', but it is not very informative.
    sig2 = 1.0 / prec
    sig = sqrt(sig2)
} "

data_jags = as.list(Anscombe)

params = c("b") # define model params to estimate 

inits = function() { # inits to get function 
    inits = list("b"=rnorm(2,0.0, 100.0))
}

mod1 = jags.model(textConnection(mod1_string), data=data_jags, inits=inits, n.chains=3)
update(mod1, 1e4) # burn-in
dic.samples(mod1, n.iter = 1e5)
#mod1_sim = coda.samples(model=mod1,variable.names=params,n.iter=1e5)
#mod1_csim = as.mcmc(do.call(rbind, mod2_sim)) # combine multiple chains

In [ ]:
library("rjags")

mod2_string = " model {
    for (i in 1:length(education)) {
        education[i] ~ dnorm(mu[i], prec)
        mu[i] = b0 + b[1]*income[i] + b[2]*young[i] + b[3]*income[i]*young[i]
    }
    
    b0 ~ dnorm(0.0, 1.0/1.0e6)
    for (i in 1:3) {
        b[i] ~ dnorm(0.0, 1.0/1.0e6)
    }
    
    prec ~ dgamma(1.0/2.0, 1.0*1500.0/2.0)
    ## Initial guess of variance based on overall
    ## variance of education variable. Uses low prior
    ## effective sample size. Technically, this is not
    ## a true 'prior', but it is not very informative.
    sig2 = 1.0 / prec
    sig = sqrt(sig2)
} "

data_jags = as.list(Anscombe)

params = c("b") # define model params to estimate 

inits = function() { # inits to get function 
    inits = list("b"=rnorm(3,0.0, 100.0))
}

mod2 = jags.model(textConnection(mod2_string), data=data_jags, inits=inits, n.chains=3)
update(mod2, 1e4) # burn-in
dic.samples(mod2, n.iter = 1e5)
#mod1_sim = coda.samples(model=mod1,variable.names=params,n.iter=1e5)
#mod1_csim = as.mcmc(do.call(rbind, mod2_sim)) # combine multiple chains

In [ ]:
plot(mod_sim)